In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
train=datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())
dataset=DataLoader(train,32)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 487kB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.84MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.07MB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
class ImageClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.model=nn.Sequential(
        nn.Conv2d(1,32,(3,3)),
        nn.ReLU(),
        nn.Conv2d(32,64,(3,3)),
        nn.ReLU(),
        nn.Conv2d(64,64,(3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*(28-6)*(28-6),10)
    )
  def forward(self,x):
    return self.model(x)


In [ ]:
clf=ImageClassifier().to('cpu')
opt=Adam(clf.parameters(), lr=1e-3)
loss_function=nn.CrossEntropyLoss()

In [ ]:
if __name__=="__main__":
  for epoch in range(10): # 10 epochs
    for batch in dataset:
      x,y=batch   # Unpacking the data
      x,y=x.to('cpu'), y.to('cpu')
      yhat=clf(x)
      loss=loss_function(yhat,y)

      # Apply Backpropogation
      opt.zero_grad()
      loss.backward()
      opt.step()
    print(f"Epoch:{epoch} loss is {loss.item()}")

Epoch:0 loss is 0.028183335438370705
Epoch:1 loss is 0.009128108620643616
Epoch:2 loss is 0.00046764855505898595
Epoch:3 loss is 1.4345516319735907e-05
Epoch:4 loss is 8.057095692493021e-05
Epoch:5 loss is 2.177214264520444e-05
Epoch:6 loss is 0.00015796742809470743
Epoch:7 loss is 0.00016813729598652571
Epoch:8 loss is 1.238115510204807e-05
Epoch:9 loss is 0.00021338736405596137


In [ ]:
test=datasets.MNIST(root='data',download=True,train=False,transform=ToTensor())
test_data=DataLoader(test,32)

In [ ]:
clf.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_data:
        images, labels = images.to('cpu'), labels.to('cpu')
        outputs = clf(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 98.28%
